# Import Modules

In [1]:
import os

os.environ["WANDB_CONSOLE"] = "off"

In [2]:
from datetime import datetime
import multiprocessing

from rnn.models import IRNN
import torch
from torch import nn
from torch.nn import ReLU
from torch.nn import MSELoss
from torch.optim import RMSprop, Adam
from torch.utils.data import DataLoader
import wandb

from data.utils import SequenceGenerator
from utils.config import SETTINGS

In [3]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

# Construct Model

In [4]:
CURRENT_SETTINGS = SETTINGS["SEQ_150"]

In [5]:
class RNNModel(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super().__init__()
        embedding_size = 2
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = IRNN(
            embedding_size,
            hidden_size,
            num_layers,
            nonlinearity="relu",
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # x.shape --> (batch_size, sequence_len, hidden_size)
        output, hidden = self.rnn(x)
        output = self.fc(output)
        output = output[:, -1, :]
        return output

In [6]:
class PLModel(pl.LightningModule):
    def __init__(self, hidden_size, num_layers):
        super().__init__()
        self.model = RNNModel(hidden_size, num_layers)
        self.criterion = MSELoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        input_, target = batch
        output = self.model(input_)
        loss = self.criterion(output, target)
        self.log("training_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)

    def configure_optimizers(self):
        # optimizer = torch.optim.SGD(self.parameters(), lr=CURRENT_SETTINGS["LR"])
        optimizer = torch.optim.Adam(self.parameters(), lr=CURRENT_SETTINGS["LR"])
        return optimizer

# Initiate Model and DataLoaders

In [7]:
model = PLModel(hidden_size=100, num_layers=1)

In [8]:
train_dataset = SequenceGenerator(CURRENT_SETTINGS["LEN"], 100_000)
train_data_loader = DataLoader(
    train_dataset, 16, num_workers=multiprocessing.cpu_count()
)
test_dataset = SequenceGenerator(CURRENT_SETTINGS["LEN"], 10_000)
test_data_loader = DataLoader(test_dataset, 32, num_workers=multiprocessing.cpu_count())

In [9]:
# for i in model.named_parameters():
#     print(i[0], i[1])

# Train

In [10]:
# wandb.finish()

In [11]:
torch.cuda.is_available()

True

In [12]:
if torch.cuda.is_available():
    trainer = pl.Trainer(
        gradient_clip_val=CURRENT_SETTINGS["GC"], accelerator="gpu", devices=1,
        max_epochs=440, log_every_n_steps=10,
        logger=WandbLogger(f"irnn-{CURRENT_SETTINGS['LEN']}-{datetime.utcnow()}", "./logs", project="othoz")
    )
else:
    trainer = pl.Trainer(
        gradient_clip_val=CURRENT_SETTINGS["GC"], max_epochs=440,
        log_every_n_steps=10,
        logger=WandbLogger(f"irnn-{CURRENT_SETTINGS['LEN']}-{datetime.utcnow()}",
                           "./logs", project="othoz")
    )


wandb: Currently logged in as: mdmmn378. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(model, train_data_loader, test_data_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type     | Params
---------------------------------------
0 | model     | RNNModel | 10.5 K
1 | criterion | MSELoss  | 0     
---------------------------------------
10.5 K    Trainable params
0         Non-trainable params
10.5 K    Total params
0.042     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=440` reached.


# Validate Manually

In [15]:
for i in test_data_loader:
    break

with torch.no_grad():
    print(model(i[0][:10].cuda()))
print(i[1][:10])